In [ ]:
import json
import threading
from tqdm import tqdm
import time
import random
from datetime import datetime, timedelta

class TemperatureSensor:
    """Simulates a temperature sensor."""
    
    def __init__(self, initial_temperature=None):
        self.temperature = initial_temperature

    def read_temperature(self, last_refill_time):
        """
        Simulate temperature reading.
        If the last refill was within the last minute, the sensed temperature will be higher.
        """
        if last_refill_time and datetime.now() < last_refill_time + timedelta(seconds=10):
            self.temperature = round(random.uniform(4.1, 25.0), 2)
        else:
            self.temperature = round(random.uniform(0.0, 4.0), 2)
        return self.temperature

class FloatSwitch:
    """Simulates a float switch sensor to monitor the remaining quantity."""
    
    def __init__(self):
        self.quantity = 100
        self.left_quantity = 100.0
        self.maintenance = "Not required"
        
    def read_quantity(self, ml):
        """
        Update the remaining quantity and return the updated value.
        Trigger maintenance warning if necessary.
        """
        erogated_qty = ((ml / 10) / self.quantity) * 100
        self.left_quantity -= erogated_qty
        self.maintenance = "Required" if self.left_quantity <= 10 else "Not required"
        return round(self.left_quantity, 2)

class Cocktail:
    """Represents a cocktail with its ingredients and preparation steps."""
    
    def __init__(self, name, ingredients, steps):
        self.name = name
        self.ingredients = ingredients
        self.steps = steps

    def show(self):
        """Display cocktail details."""
        print(f"Cocktail: {self.name}")
        print("Ingredients:")
        for ingredient, details in self.ingredients.items():
            print(f"  {ingredient}: {details['quantity']}ml") #at {details['optimal_temp_C']}°C"
        print()

class Pump:
    """Represents a pump that handles dispensing of cocktail ingredients."""
    def __init__(self, id, ingredient, temperature, maintenance, cocktails, temperature_sensor, float_switch, last_refill_time):
        self.id = id
        self.ingredient = ingredient
        self.temperature = temperature
        self.maintenance = maintenance
        self.cocktails = cocktails
        self.temperature_sensor = temperature_sensor
        self.float_switch = float_switch
        self.last_refill_time = last_refill_time

    def display_status(self):
        """Display the current status of the pump."""
        temp = self.temperature_sensor.read_temperature(self.last_refill_time)
        quantity = self.float_switch.left_quantity
        print(f"Pump Number: {self.id},\nIngredient: {self.ingredient},\nCurrent Temperature: {temp}°C,\nRemaining Quantity: {round(quantity,2)}%,\nMaintenance Needed: {self.float_switch.maintenance},\nConfigured for: {self.cocktails}\n")

    def refill(self):
        """Refill the pump and reset its quantity."""
        print(f"Refilling {self.ingredient}...")
        self.float_switch.left_quantity = 0  # Start from empty to show refill progress
        refill_duration = 10  # seconds
        steps = 100
        step_time = refill_duration / steps

        # Simulate the refilling process with a progress bar
        for _ in tqdm(range(steps), desc=f"Refilling {self.ingredient}", unit="step"):
            time.sleep(step_time)
        
        self.float_switch.left_quantity = 100
        self.last_refill_time = datetime.now()
        self.temperature_sensor.read_temperature(self.last_refill_time)
        print(f"{self.ingredient} refilled. Current quantity: {self.float_switch.left_quantity}%. Current temperature: {self.temperature_sensor.read_temperature(self.last_refill_time)}°C\n" )
    
    def wait_for_optimal_temperature(self, optimal_temp):
        """
        Check if the ingredient has reached the optimal temperature.
        Return False if the temperature is still above the optimal value.
        """
        current_temp = self.temperature_sensor.read_temperature(self.last_refill_time)
        if current_temp > optimal_temp:
            print(f"The {self.ingredient} is still above its optimal temperature. Please wait a few minutes or choose another cocktail.")
            return False
        return True

    def erogate(self, ingredient, ml, optimal_temp, required_qty_percent):
        """
        Dispense the specified amount of ingredient if the temperature is optimal and the quantity is enough.
        """
        if not self.wait_for_optimal_temperature(optimal_temp):
            return

        if self.float_switch.left_quantity < required_qty_percent:
            self.refill()
            if not self.wait_for_optimal_temperature(optimal_temp):
                return

        print(f"Erogating {ml}ml of {ingredient}...")
        flow_rate = 300  # ml per minute
        total_time_seconds = (ml / flow_rate) * 60
        steps = 100
        step_time = total_time_seconds / steps

        for _ in tqdm(range(steps), desc=f"Erogating {ingredient}", unit="step"):
            time.sleep(step_time)

        print(f"Remaining quantity of {ingredient}: {self.float_switch.read_quantity(ml)}%\n") 

class Smartender:
    """Main class to handle the Smartender operations."""
    
    def __init__(self, filename1):
        """
        Initialize Smartender with a JSON file containing cocktail data.
        """
        self.filename1 = filename1
        self.available_cocktails = []
        self.selected_cocktails = []
        self.selected_ingredients = []
        self.active_pumps = []
        self.countdown_thread = None
        self.countdown_event = threading.Event()

        self.load_cocktails()

    def load_cocktails(self):
        """Load cocktails from a JSON file."""
        try:
            with open(self.filename1, 'r') as file:
                data = json.load(file)
                for name, details in data.items():
                    self.available_cocktails.append(Cocktail(name, **details))
        except (FileNotFoundError, json.JSONDecodeError) as e:
            print(f"An error occurred while loading cocktails: {e}")

    def display_pump_status(self):
        """Display status of all active pumps."""
        for pump in self.active_pumps:
            pump.display_status()

    def show_cocktails(self, cocktails):
        """Display list of available cocktails."""
        print("\nAvailable Cocktails:\n")
        for cocktail in cocktails:
            cocktail.show()

    def get_user_input(self, prompt, quit_option='q'):
        """Get user input and handle quitting."""
        user_input = input(prompt).strip()
        if user_input.lower() == quit_option:
            return None
        return user_input

    def configure(self):
        """Handles Smartender configuration."""
        print("Welcome to your SmarTender!\n")
        while len(self.selected_ingredients) < 20:
            user_input = self.get_user_input("What cocktails do you want to make? Choose one or more from the available options: q to quit\t")
            if user_input is None:
                break
            self.add_cocktail(user_input)

        self.setup_pumps()

    def add_cocktail(self, user_input):
        """Add a selected cocktail to the Smartender menu."""
        for cocktail in self.available_cocktails:
            if user_input.lower() == cocktail.name.lower():
                self.selected_cocktails.append(cocktail)
                for ingredient in cocktail.ingredients:
                    if ingredient not in self.selected_ingredients:
                        self.selected_ingredients.append(ingredient)
                print(f"{cocktail.name} added to your Smartender\n")

    def setup_pumps(self):
        """Set up pumps for selected ingredients and cocktails."""
        id = 0
        for cocktail in self.selected_cocktails:
            for ingredient, details in cocktail.ingredients.items():
                if not self.pump_exists(ingredient):
                    self.active_pumps.append(Pump(id, ingredient, details['temperature'], None, [cocktail.name], TemperatureSensor(), FloatSwitch(), datetime.now()))
                    id += 1
                else:
                    self.update_pump_cocktails(ingredient, cocktail.name)
        print("Pumps successfully configured!\n")

    def pump_exists(self, ingredient):
        """Check if a pump for the ingredient already exists."""
        return any(pump.ingredient == ingredient for pump in self.active_pumps)

    def update_pump_cocktails(self, ingredient, cocktail_name):
        """Update the list of cocktails for an existing pump."""
        for pump in self.active_pumps:
            if pump.ingredient == ingredient:
                pump.cocktails.append(cocktail_name)

    def countdown_timer(self):
        """Start a countdown timer to simulate ingredients cooling waiting time."""
        self.countdown_event.clear()
        for i in range(10 * 60, -10, -60):
            if self.countdown_event.is_set():
                return
            mins, secs = divmod(i, 60)
            timeformat = '{:02d}:{:02d}'.format(mins, secs)
            print(timeformat, end='\r')
            time.sleep(1)

    def wait_for_ingredients(self, pumps, optimal_temps):
        """Wait until all ingredients are at their optimal temperature."""
        while True:
            all_optimal = all(
                pump.temperature_sensor.read_temperature(pump.last_refill_time) <= optimal_temp
                for pump, optimal_temp in zip(pumps, optimal_temps)
            )

            if all_optimal:
                print("All ingredients have reached their optimal temperatures.")
                return

            print("Some ingredients are still above their optimal temperatures. Please wait a few minutes or choose another cocktail.")
            if not self.countdown_thread or not self.countdown_thread.is_alive():
                self.countdown_thread = threading.Thread(target=self.countdown_timer)
                self.countdown_thread.start()

            user_input = self.get_user_input("Press 'b' to choose another cocktail or wait for the countdown: ")
            if user_input.lower() == 'b':
                self.countdown_event.set()
                self.countdown_thread.join()
                return

    def make_cocktail(self):
        """Prepare the selected cocktail."""
        user_input = self.get_user_input("What cocktail do you want to drink? Choose one or more from the available options\t")
        if user_input is None:
            return

        for cocktail in self.selected_cocktails:
            if user_input.lower() == cocktail.name.lower():
                print(f"\nYou chose {cocktail.name}!\n")

                ingredients_to_cool = []
                optimal_temps = []

                for ingredient, details in cocktail.ingredients.items():
                    ingredient_name = ingredient
                    ml = details['quantity']
                    optimal_temp = details['optimal_temp_C']

                    for pump in self.active_pumps:
                        if pump.ingredient == ingredient_name:
                            required_qty_percent = ((ml / 10) / pump.float_switch.quantity) * 100

                            if pump.float_switch.left_quantity < required_qty_percent:
                                print(f"Not enough {ingredient_name} left to make {cocktail.name}. Refilling the pump.")
                                pump.refill()

                            if pump.temperature_sensor.read_temperature(pump.last_refill_time) > optimal_temp:
                                ingredients_to_cool.append(pump)
                                optimal_temps.append(optimal_temp)

                if ingredients_to_cool:
                    self.wait_for_ingredients(ingredients_to_cool, optimal_temps)
                    return

                for ingredient, details in cocktail.ingredients.items():
                    ingredient_name = ingredient
                    ml = details['quantity']
                    optimal_temp = details['optimal_temp_C']
                    for pump in self.active_pumps:
                        if pump.ingredient == ingredient_name:
                            required_qty_percent = ((ml / 10) / pump.float_switch.quantity) * 100
                            pump.erogate(ingredient_name, ml, optimal_temp, required_qty_percent)

                print("Your cocktail is ready. Enjoy!")
                return

if __name__ == "__main__":
    smartender = Smartender('cocktails.json')
    smartender.show_cocktails(smartender.available_cocktails)
    smartender.configure()

    while True:
        smartender.show_cocktails(smartender.selected_cocktails)
        smartender.make_cocktail()



Available Cocktails:

Cocktail: Margarita
Ingredients:
  Tequila: 50ml
  Triple Sec: 20ml
  Lime Juice: 15ml

Cocktail: Martini
Ingredients:
  Gin or Vodka: 50ml
  Dry Vermouth: 10ml

Cocktail: Manhattan
Ingredients:
  Whiskey: 50ml
  Sweet Vermouth: 20ml
  Bitters: 1ml

Cocktail: Daiquiri
Ingredients:
  Rum: 50ml
  Lime Juice: 15ml
  Simple Syrup: 10ml

Cocktail: Gin and Tonic
Ingredients:
  Gin: 50ml
  Tonic Water: 100ml

Cocktail: Gin Lemon
Ingredients:
  Gin: 50ml
  Lemon Soda: 100ml

Cocktail: Mojito
Ingredients:
  Rum: 50ml
  Lime Juice: 15ml
  Simple Syrup: 10ml
  Soda Water: 100ml
  Mint: 10ml

Cocktail: Whiskey Sour
Ingredients:
  Whiskey: 50ml
  Lemon Juice: 15ml
  Simple Syrup: 10ml

Cocktail: Tom Collins
Ingredients:
  Gin: 50ml
  Lemon Juice: 15ml
  Simple Syrup: 10ml
  Soda Water: 100ml

Cocktail: Cosmopolitan
Ingredients:
  Vodka: 40ml
  Triple Sec: 15ml
  Cranberry Juice: 30ml
  Lime Juice: 15ml

Cocktail: Negroni
Ingredients:
  Gin: 30ml
  Sweet Vermouth: 30ml
  Campa

What cocktails do you want to make? Choose one or more from the available options: q to quit	 negroni


Negroni added to your Smartender



What cocktails do you want to make? Choose one or more from the available options: q to quit	 q


Pumps successfully configured!


Available Cocktails:

Cocktail: Negroni
Ingredients:
  Gin: 30ml
  Sweet Vermouth: 30ml
  Campari: 30ml



...

Remaining quantity of Gin: 97.0%

Erogating 30ml of Sweet Vermouth...


Erogating Sweet Vermouth: 100%|█████████████| 100/100 [00:06<00:00, 15.47step/s]


Remaining quantity of Sweet Vermouth: 97.0%

Erogating 30ml of Campari...


Erogating Campari: 100%|████████████████████| 100/100 [00:06<00:00, 15.49step/s]


Remaining quantity of Campari: 97.0%

Your cocktail is ready. Enjoy!

Available Cocktails:

Cocktail: Negroni
Ingredients:
  Gin: 30ml
  Sweet Vermouth: 30ml
  Campari: 30ml

